In [1]:
import requests
import gradio as gr
import json
from getpass import getpass
from huggingface_hub import login

In [ ]:
login(token="#change the token into your hf token ") #change the token into your hf token 
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
headers = {"Authorization": f"Bearer {'#change the token into your hf token '}"}

In [3]:
def check_model_status(url):
    try:
        response = requests.post(url, headers=headers, json={"inputs": "test"})
        return "Model is ready" if response.status_code == 200 else f"Status: {response.status_code}"
    except Exception as e:
        return f"Connection error: {str(e)}"

In [20]:

def generate_response(task, user_input):
    try:
        if task == "FAQ":
            prompt = f"""Answer these common questions based on examples:

Q: What is your return policy?
A: We accept returns within 30 days of purchase.

Q: How do I track my order?
A: Use the tracking number in your confirmation email.

Q: {user_input}
A: """
        elif task == "Math":
            prompt = f"""Solve this problem step-by-step. 
Example: 
Problem: If Jane has 3 apples and buys 5 more, how many does she have?
Solution: 
1. Start with initial apples: 3
2. Add purchased apples: 3 + 5 = 8
3. Final answer: 8

Problem: {user_input}
Solution:"""
        elif task == "Joke":
            prompt = f"""Generate a joke about {user_input} formatted as JSON with setup and punchline keys.

Example:
{{
    "setup": "Why did the programmer quit their job?",
    "punchline": "Because they didn't get arrays!"
}}

New joke:"""

        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 200,
                "temperature": 0.7,
                "return_full_text": False
            }
        }

        response = requests.post(API_URL, headers=headers, json=payload)
        output = json.loads(response.content.decode("utf-8"))

        if "error" in output:
            return f"⚠️ Model error: {output['error']}"

        answer = output[0]['generated_text'].strip()

        # Trim repeated examples
        if task == "Math":
            parts = answer.split("Problem:")
            for part in parts:
                if user_input.strip() in part:
                    return "Problem:" + part.strip()
            return parts[0].strip()

        return answer

    except Exception as e:
        return f"❌ Error: {str(e)}"

In [21]:

def chat(task, user_input):
    if not user_input:
        return gr.update(), gr.update()
    response = generate_response(task, user_input)
    chat_history.append(f"**You:** {user_input}\n**Bot:** {response}")
    return "\n\n---\n\n".join(chat_history), response

def clear():
    chat_history.clear()
    return "", ""

In [22]:
with gr.Blocks(theme=gr.themes.Monochrome(), css="footer {display: none;}") as demo:
    gr.Markdown("""
    # 🤖 Multi-Task Chatbot
    _Your smart assistant for FAQs, Math help, and Jokes!_
    """)

    with gr.Row():
        task = gr.Dropdown(["FAQ", "Math", "Joke"], label="Choose Task", interactive=True)
        user_input = gr.Textbox(lines=1, placeholder="Type your message here...", label="Input")

    with gr.Row():
        submit_btn = gr.Button("🚀 Send", variant="primary")
        clear_btn = gr.Button("🧹 Clear Chat")

    gr.Markdown("---")

    with gr.Row():
        chat_output = gr.Markdown(label="📜 Chat History", elem_id="chat_history")
        latest_response = gr.Textbox(label="💬 Latest Response", lines=8, interactive=False)

    submit_btn.click(chat, inputs=[task, user_input], outputs=[chat_output, latest_response])
    clear_btn.click(clear, outputs=[chat_output, latest_response])


In [23]:
demo.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
